In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
def cardinalidad(df_in, umbral_categoria, umbral_countinua):
    cardinalidad = [df_in[col].nunique() for col in df_in.columns]
    cardinalidad_por = [df_in[col].nunique()/len(df_in[col]) for col in df_in.columns]
    dict_df = {"nombre_variable": df_in.columns, "valores_unicos": cardinalidad, "cardinalidad": cardinalidad_por}
    nuevo_df = pd.DataFrame(dict_df)
    nuevo_df["tipo_sugerido"] = "Categórica"
    nuevo_df.loc[nuevo_df["valores_unicos"] == 2, "tipo_sugerido"] = "Binaria"
    nuevo_df.loc[nuevo_df["valores_unicos"] >= umbral_categoria, "tipo_sugerido"] = "Numerica Discreta"
    nuevo_df.loc[nuevo_df["cardinalidad"] >= umbral_countinua, "tipo_sugerido"] = "Numerica Continua"
    return nuevo_df

In [14]:
def get_features_cat_regresssion(df, target_col, p_value = 0.05):
    from scipy.stats import mannwhitneyu
    from scipy.stats import f_oneway
    lista_cat = []
    for col in df:
        values_no = df[col].nunique()
        if values_no == 2:
            value_1 = df[col].unique()[0]
            value_2 = df[col].unique()[1]
            group_a = df.loc[df[col] == value_1, target_col]
            group_b = df.loc[df[col] == value_2, target_col]
            _, p_val = mannwhitneyu(group_a, group_b)
            print(f"Para {target_col} y {col} el p-value es: {p_val}")
            print(p_val)
            if p_val < p_value:
                lista_cat.append(col)
        elif values_no > 2 and values_no <= 10:
            groups = df[col].unique()
            target_values_x_group = [df.loc[df[col] == group, target_col] for group in groups]
            _, p_val = f_oneway(*target_values_x_group)
            print(f"Para {target_col} y {col} el p-value es: {p_val}")
            if p_val < p_value:
                lista_cat.append(col)
    return lista_cat
    

In [5]:
df_titanic = pd.read_csv("./data/titanic.csv")
df_titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [15]:
get_features_cat_regresssion(df_titanic, "age")

Para age y survived el p-value es: nan
nan
Para age y pclass el p-value es: nan
Para age y sex el p-value es: nan
nan
Para age y sibsp el p-value es: nan
Para age y parch el p-value es: nan
Para age y embarked el p-value es: nan
Para age y class el p-value es: nan
Para age y who el p-value es: nan
Para age y adult_male el p-value es: nan
nan
Para age y deck el p-value es: nan
Para age y embark_town el p-value es: nan
Para age y alive el p-value es: nan
nan
Para age y alone el p-value es: nan
nan


[]

In [16]:
from scipy.stats import mannwhitneyu
from scipy.stats import f_oneway

In [ ]:
groups = df_titanic.unique()
target_values_x_group = [df.loc[df[col] == group, targ] for group in groups]
_, p_val = f_oneway(*target_values_x_group)

In [29]:
df_titanic.dropna(inplace=True)

In [30]:
value_1 = df_titanic["survived"].unique()[0]
value_2 = df_titanic["survived"].unique()[1]
group_a = df_titanic.loc[df_titanic["survived"] == value_1, "age"]
group_b = df_titanic.loc[df_titanic["survived"] == value_2, "age"]
p, p_val = mannwhitneyu(group_a, group_b)
print(p_val)

0.0006641019760370105


In [28]:
value_1 = df_titanic["survived"].unique()[0]
value_1
group_a = df_titanic.loc[df_titanic["survived"] == value_1]["age"]
group_a
group_b = df_titanic.loc[df_titanic["survived"] == value_2, "age"]
group_a

0      22.0
4      35.0
5       NaN
6      54.0
7       2.0
       ... 
884    25.0
885    39.0
886    27.0
888     NaN
890    32.0
Name: age, Length: 549, dtype: float64